# TEDS End-to-End Simulation for TANGO-Carbon

### This notebook includes a first processing chain for TANGO Carbon. The chain is setup for consistent processing that is outlined in the following diagram:  

![title](./tango_e2es_Carbon_chain_PAM.png)

## Reference
+ ATBD
+ User manual

## Initialization

#### 1. Setup the working environment

In [1]:
import sys
import os
import yaml

# define  path to search for module
sys.path.append("/home/jochen/TANGO_E2ES/EndtoEndProject/end_to_end/")
os.chdir("/home/jochen/TANGO_E2ES/EndtoEndProject/end_to_end/examples/exp1.0_full_chain/")

#### 2. import TEDS modules

In [2]:
from teds.gm import geometry_module
from teds.sgm import geoscene_generation
from teds.sgm import Carbon_radiation_scene_generation
from teds.im import run_instrument_model
from teds.l1al1b import run_l1al1b
from teds.siml1b import simplified_instrument_model_and_l1b_processor
from teds.l1l2.l1bl2 import level1b_to_level2_processor_RTorCH4
from teds.pam import pam_co2

#### 3. Select the target scene named after the targeted power plant 
Currently, four reference scenes can be used: (1) Matimba, (2) Jenschwalde, (3) Belchatow, (4) Lipetsk.


In [3]:
#station_name = 'Matimba'
station_name = 'Jaenschwalde'
#station_name = 'Belchatow'
#station_name = 'Lipetsk'

In [4]:
proc_gm     = False ; pam_gm = False
proc_sgmgps = False ; pam_sgmgps = False
proc_sgmrad = False ; pam_sgmrad = False
proc_im     = False ; pam_im = False
proc_l1al1b = False ; pam_l1b = True
proc_siml1b = False  
proc_l1bl2  = False ; pam_l2 = False

## Geometry Model

#### 1. runing the module

In [5]:
if(proc_gm):
    #configureation file
    gm_config= yaml.safe_load(open('./settings/gm_config_stations.yaml'))
    #select the list for the selected target
    gm_config['orbit'] = gm_config[station_name]['orbit']
    gm_config['sensors'] = gm_config[station_name]['sensors']
    geometry_module(gm_config)

#### 2. Visualization of the GM output

For plt_options use 'pixel' to show the geolocation of the pixel center points, and 'geometry' to show solar zenith angle, viewing zenith angle, solar azimuth angle, and viewing azimuth angle.

In [6]:
if(pam_gm):
    gm_config= yaml.safe_load(open('./settings/gm_config_stations.yaml'))
    filen = gm_config['io_files']['output_gm']
    plt_option = 'geoloc'
    pam_co2.pam_gm_Tango_Carbon(filen, station_name, plt_option)

## Scene Generatior Module (SGM): Geo-Physical Scene

### 1. Run the module

In [7]:
if(proc_sgmgps):
    sgmgps_config= yaml.safe_load(open('./settings/geosgm_config_stations.yaml'))
    #adjust target specific configs
    sgmgps_config['io_files']['input_s2'] = sgmgps_config[station_name]['input_s2']
    sgmgps_config['io_files']['meteo'] = sgmgps_config[station_name]['meteo']

    geoscene_generation(sgmgps_config)

### 2. Vizualization of SGM-GPS output

plt_options is used to configure the output. 
* The input 'albedo Bxx' specifies the spectral band to be plotted. Here, xx is the band specification. Only bands can be plotted that are given in the input data (e.g. B08 is not available but the more norrow band B08A can be plotted). 
* The input 'Xxx' specifies the trace gas. Here, xx specifies the gas and can be CO2, NO, NO2, CH4, or H2O

In [8]:
if(pam_sgmgps):
    sgmgps_config= yaml.safe_load(open('./settings/geosgm_config_stations.yaml'))
    filen = sgmgps_config['io_files']['output_geo']
    plt_option = 'XCO2'
    plt_option = 'albedo B11'
    pam_co2.pam_sgm_gps(filen, station_name, plt_option)

## Scene Generator Module (SGM): Radiation Scene

### 1. Execute SGM Radiation scene

In [9]:
if(proc_sgmrad):
    sgmrad_config= yaml.safe_load(open('./settings/radsgm_config.yaml'))
    Carbon_radiation_scene_generation(sgmrad_config)

### 2. Visualization
Two panels are created. First a map of the radiometric scene for the wavelength wavel. On this map up to four pixels can be chosen using the tuple ialt_iact. The corresponding spectra are dipicted in the right panel.

In [10]:
if(pam_sgmrad):
    sgmrad_config= yaml.safe_load(open('./settings/radsgm_config.yaml'))
    filen_rad = sgmrad_config['io_files']['output_rad']
    wavel = 1600.0
    ialt_iact = [[50,50], [100,3], [105,79], [0,1]]
    pam_co2.pam_sgm_rad(filen_rad, station_name, wavel, ialt_iact)

## Instrument Model (IM)

some text

### 1. Executing the model

In [11]:
if(proc_im):
    im_config= yaml.safe_load(open('./settings/im_config.yaml'))
    run_instrument_model(im_config)

### 2. Plotting detector images

The generated L0 data contain a series of detector images. The index ialt points to the image.

In [12]:
if(pam_im):
    filen_rad = im_config['io']['l1a']
    ialt = 5
    title = station_name + f' image no. {ialt}'
    pam_co2.pam_im(filen_rad, ialt, title)

## L1A to L1B Processor

The L1A-L1B processor is a module for the Payload Data Ground Segement. It calibrated the simulated detector images.

### 1. Execute L1A-L1B data processing

In [13]:
if(proc_l1al1b):
    l1al1b_config= yaml.safe_load(open('./settings/l1al1b_config.yaml'))
    run_l1al1b(l1al1b_config)

### 2. Simplified L1B generator

For some studies, it makes sense to replace the IM and the L1A-L1B processor by a simple L1B generator that simulates a L1B product from the SGM simulations. It applies a spectral convolution and a two-parameter SNR model to the spectra and allocates the GM geolocation to the data product.

In [14]:
if(proc_siml1b):
    siml1b_config= yaml.safe_load(open('./settings/siml1b_config.yaml'))
    simplified_instrument_model_and_l1b_processor(siml1b_config)

### 3. Comparision with SGM-RAD output

plt_options = spectrum, histo

In [16]:
if(pam_l1b):
    siml1b_config= yaml.safe_load(open('./settings/siml1b_config.yaml'))
    filen_l1b = siml1b_config['io_files']['output_l1b']
    filen_sgmrad = siml1b_config['io_files']['input_sgm']
    isrf_config={}
    isrf_config['type'] = 'Gaussian'  #type of ISRF, currently only Gaussian or generalized_normal
    isrf_config['fwhm'] = 0.45        #fwhm [nm]
    plt_options = 'spectrum'
    ialt,iact =(26,44)                #onoly needed in case of plt_options = spectrum
    plt_options = 'histo'
    plt_ext = False
    pam_co2.pam_l1b(filen_l1b, filen_sgmrad, isrf_config, plt_options, ialt, iact, plt_ext)

===> 6784000


IndexError: Index cannot be multidimensional

## Level 1B to Level 2 $\rm CO_2$ and $\rm CH_4$ Proxy Retrieval

In [ ]:
if(proc_l1bl2):
    l1bl2_config= yaml.safe_load(open('./settings/l1bl2_config.yaml'))
    level1b_to_level2_processor_RTorCH4(l1bl2_config)

In [ ]:
if(pam_l2):
    %matplotlib inline
    l1bl2_config= yaml.safe_load(open('./settings/l1bl2_config.yaml'))
    filen = l1bl2_config['io_files']['output_l2']
    pam_co2.pam_l2(filen, station_name)

